In [1]:
import requests
from POP import PromptFunction
#from POP.utils import load_prompt, get_text_snapshot
from openai import OpenAI
from dotenv import load_dotenv
from os import getenv
import json

## Perplexity

In [10]:
url = "https://api.perplexity.ai/chat/completions"

payload = {
    "model": "sonar",
    "messages": [
        {
            "role": "system",
            "content": "Be precise and concise."
        },
        {
            "role": "user",
            "content": "Why is winter cold?"
        }
    ],
    #"max_tokens": "Optional",
    "temperature": 0.2,
    "top_p": 0.9,
    #"search_domain_filter": ["perplexity.ai"],
    "return_images": False,
    "return_related_questions": False,
    "search_recency_filter": "month",
    "top_k": 0,
    "stream": False,
    "presence_penalty": 0,
    "frequency_penalty": 1,
    "response_format": None
}
headers = {
    "Authorization": f"Bearer {getenv('PERPLEXITY_API_KEY')}",
    "Content-Type": "application/json"
}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)

{"id": "a021e45e-0446-49d0-855a-a15021a17964", "model": "sonar", "created": 1740437547, "usage": {"prompt_tokens": 10, "completion_tokens": 130, "total_tokens": 140}, "citations": ["https://www.dw.com/en/earth-is-warming-so-why-is-it-so-cold/a-71661055", "https://www.climate.gov/news-features/blogs/polar-vortex/cold-air-its-probably-not-polar-vortex", "https://www.nbcwashington.com/weather/science-4-everyone-how-do-bugs-survive-during-the-cold-winter-months/3850831/"], "object": "chat.completion", "choices": [{"index": 0, "finish_reason": "stop", "message": {"role": "assistant", "content": "Winter cold is primarily due to Earth's tilt away from the Sun during this season, reducing the amount of solar energy received by the Northern Hemisphere. Additionally, cold air masses from polar regions can move towards lower latitudes, bringing frigid temperatures. In recent years, phenomena like the **polar vortex** have been linked to extreme cold snaps. The polar vortex is a band of winds that

In [11]:
client_perplexity = OpenAI(base_url="https://api.perplexity.ai", api_key=getenv("PERPLEXITY_API_KEY"))

messages = [
                {
                    "role": "system",
                    "content": (
                        "You are an artificial intelligence assistant and you need to "
                        "engage in a helpful, detailed, polite conversation with a user."
                    ),
                },
                {
                    "role": "user",
                    "content": "What's the weather for east america? Include MA, NY, and PA.",
                },
            ]
response = client_perplexity.chat.completions.create(model="sonar-pro", messages=messages)
answer = response.choices[0].message.content
citations = response.citations  # usually a list of strings
print("Answer:\n", answer)
print("Citations:\n", citations)

Answer:
 Based on the search results, here's a summary of the weather forecast for eastern America, focusing on Massachusetts (MA), New York (NY), and Pennsylvania (PA) for February 2025:

Massachusetts:
February 2025 in Massachusetts is expected to be cooler than usual, with temperatures averaging around 33°F, which is about 5°F below the typical seasonal average[1]. The forecast for February 24 shows partly cloudy conditions with a high of 40°F and a low of 30°F[5]. Overall, the month is predicted to be quite snowy, with an average of 15 snowy days throughout February[1].

New York:
While specific data for New York is not provided in the search results, we can infer similar conditions to nearby states. The weather is likely to be cold, with temperatures potentially ranging from the low 30s to low 40s Fahrenheit.

Pennsylvania:
In Pennsylvania, February 2025 is forecasted to be cold with temperatures ranging from 28°F to 41°F[10]. For Philadelphia specifically, the forecast for Februa

## Deepseek (Local)

In [13]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

#tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B")
#model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B")
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

# Generate a response
prompt = "What is the capital of the United States?"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_length=100)
response = tokenizer.decode(outputs[0])
print(response)

c:\Users\sgt17\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


<｜begin▁of▁sentence｜>What is the capital of the United States? I need the answer in a box.
</think>

The capital of the United States is **Washington, D.C.**<｜end▁of▁sentence｜>


## Story Outliner

In [2]:
load_prompt = PromptFunction().load_prompt
pf_outliner = PromptFunction(load_prompt("story_outliner_prompt.md"))
print(pf_outliner.prompt)

No prompt or system prompt provided.
Placeholders found: ['user_request']



## GPT Auto-JSON formatter

GPT Auto formatter is a prompt function that outputs the `fmt` accepted by `PromptFunction` objects, based on given Natural Language description. It can also be used in auto task chains.

In [3]:
prompt_path = "prompts/json_formatter_prompt.md"
#prompt_path = "prompts/openai-json_schema_generator.md"
pf_formatter = PromptFunction(sys_prompt=load_prompt(prompt_path),
                              prompt="I need help generating a JSON schema for a specific format. Can you assist me with this task? Here's the description: \n<<<user_description>>>",)

Placeholders found: ['user_description']


In [4]:
## make sure the return is a json object, we create a flexible json schema to allow any key value pairs
formatter_fmt = {
  "name": "Any_name",
  "schema": {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "additionalProperties": True,
    "properties": {}
    }
}

Here's an example to generate json schema using descriptive language, and the return with, and without the structured output formatter.

In [6]:
json_fmt = pf_formatter.execute(user_description = "'student' is a list of dictionary, each contains exactly 2 keys: 'name' and 'age'. 'name' is a string and 'age' is an integer. Add another GPA field also", 
                                model = "gpt-4o-mini",
                                fmt = formatter_fmt,
                                temp = 0.02)
json_fmt

'{\n  "name": "Student_Format",\n  "schema": {\n    "$schema": "http://json-schema.org/draft-07/schema#",\n    "type": "object",\n    "additionalProperties": false,\n    "properties": {\n      "student": {\n        "type": "array",\n        "items": {\n          "type": "object",\n          "properties": {\n            "name": {\n              "type": "string"\n            },\n            "age": {\n              "type": "integer"\n            },\n            "GPA": {\n              "type": "number"\n            }\n          },\n          "required": ["name", "age", "GPA"],\n          "additionalProperties": false\n        }\n      }\n    },\n    "required": ["student"]\n  }\n}'

In [7]:
json.loads(json_fmt)

{'name': 'Student_Format',
 'schema': {'$schema': 'http://json-schema.org/draft-07/schema#',
  'type': 'object',
  'additionalProperties': False,
  'properties': {'student': {'type': 'array',
    'items': {'type': 'object',
     'properties': {'name': {'type': 'string'},
      'age': {'type': 'integer'},
      'GPA': {'type': 'number'}},
     'required': ['name', 'age', 'GPA'],
     'additionalProperties': False}}},
  'required': ['student']}}

In [10]:
story = '''
The Fox and the Hedgehog
A fox knows many things, but a hedgehog knows one big thing.

The story
Upon crossing a river, a Fox gets his tail entangled in a bush and cannot move. While the trapped and in some dismay, a swarm of mosquitoes settle on the poor Fox and tuck into a hearty meal, unable to be swished away by the vulpine’s ensnared tail. 

‘You are in a bad way, neighbour,’ says a passing Hedgehog, seeing the Fox’s predicament. Trying to help, the Hedgehog suggests that he drive the mosquitoes away who are sucking his blood. But the wise Fox realises that by ridding him of these insects, who’ve already had their fill, would only lead to more mosquitoes arriving with a fresh appetite…and so refuses. 

The moral
Better to bear a lesser evil than to risk a greater in removing it.
'''

json_fmt = pf_formatter.execute(user_description = f"generate a JSON schema for extracting essential setups the following story, such that rebuilding it from the json can generate a story that well resemble the original story:: \n{story}", 
                                model = "gpt-4o-mini",
                                fmt = formatter_fmt,
                                temp = 0.02)
json.loads(json_fmt)

{'name': 'Story_Extraction_Schema',
 'schema': {'$schema': 'http://json-schema.org/draft-07/schema#',
  'type': 'object',
  'additionalProperties': False,
  'properties': {'title': {'type': 'string',
    'description': 'The title of the story.'},
   'characters': {'type': 'array',
    'description': 'List of characters in the story.',
    'items': {'type': 'string'}},
   'setting': {'type': 'string',
    'description': 'The setting where the story takes place.'},
   'plot': {'type': 'string',
    'description': 'A brief summary of the main events in the story.'},
   'moral': {'type': 'string',
    'description': 'The moral lesson conveyed by the story.'}},
  'required': ['title', 'characters', 'setting', 'plot', 'moral']}}

In [12]:
pf = PromptFunction('')
response = pf.execute(ADD_BEFORE=story, model="gpt-4o-mini",fmt = formatter_fmt)
story_setup = json.loads(response)
story_setup

No prompt or system prompt provided.


{'type': 'object', 'properties': {}}

In [13]:
pf = PromptFunction('')
response = pf.execute(ADD_BEFORE="The student name is John and age is 20. Unfortunately, his GPA is 2.7 ", model="gpt-4o-mini",fmt = json.loads(json_fmt))
story_setup = json.loads(response)
story_setup

No prompt or system prompt provided.


{'title': "John's Academic Struggles",
 'characters': ['John'],
 'setting': 'A university campus',
 'plot': 'John, a 20-year-old student, is facing difficulties in his academic life due to a low GPA of 2.7. He battles with time management, distractions, and a lack of motivation, which impacts his grades. With support from friends and faculty, he learns to develop better study habits and ultimately improves his academic performance.',
 'moral': "With determination and the right support, it is possible to overcome challenges and improve one's situation."}

In [14]:
pf = PromptFunction('')
pf.execute(ADD_BEFORE = "what's the weather in New York?",
           sys = "Here's the weather in New York: Rainny, 42F, and partially cloudy.",)


No prompt or system prompt provided.


"The weather in New York is rainy, with a temperature of 42°F, and it's partially cloudy."

In [15]:
response = pf.execute(ADD_BEFORE = "what's the weather in New York?",
           sys = "Here's the weather in New York: Rainny, 42F, and partially cloudy.",
           fmt = json.loads(json_fmt))
story_setup = json.loads(response)
story_setup

{'title': 'The Rainy Day in New York',
 'characters': ['Alex', 'Jamie', 'Sam'],
 'setting': 'New York City on a rainy day, 42F, partially cloudy',
 'plot': "Alex, Jamie, and Sam decide to explore New York City despite the rainy weather. They visit a cozy café, share stories, and enjoy the city's ambiance. They realize that even a rainy day can be enjoyable if spent with good friends.",
 'moral': 'Friendship can brighten even the gloomiest of days.'}

## Function calling

In [ ]:
pf_func_call_formatter = PromptFunction('''
### Task: Generate Function Calling Definitions

As an expert in creating function calling objects for auto-function invocation, your task is to generate a list of function calling definitions based on the provided user description. Follow these instructions:

1. **Use the Provided User Description**: Utilize the placeholder `user_description` to tailor the function definitions. The description should inform what functions are needed.

2. **Generate a List of Functions**: The output must be a JSON array (list) where each element is a JSON object with the following structure:
   - `"type"`: This key must have the string value `"function"`.
   - `"function"`: This is an object that must include:
     - `"name"`: A unique function name (string).
     - `"description"`: A brief description of the function's purpose (string).
     - `"parameters"`: A JSON Schema object that defines the expected input parameters. It must include:
         - `"type"`: Must be `"object"`.
         - `"properties"`: An object detailing each parameter’s type and description.
         - `"required"`: An array listing the names of required parameters.
         - `"additionalProperties"`: A boolean value (true or false) indicating whether additional properties are allowed.

3. **Output Only Valid JSON**: The final output must be valid JSON with no additional text, markdown, or explanation. It should be a JSON list of function definitions exactly as specified.

**IMPORTANT**: Your final output must be a valid JSON array of function calling definitions, where each definition includes the keys `"type"` and `"function"` (with `"name"`, `"description"`, and `"parameters"` defined as per the instructions).

### User Description

<<<user_description>>>

''')

## GPT Auto-Func Calling formatter

In [ ]:
tools = [{
    "type": "function",
    "function": {
        "name": "generate_json_schema",
        "description": (
            "Generate a JSON schema based on a user description. "
            "The output must be a valid JSON object that contains the keys 'name' and 'schema'. "
            "The 'schema' must follow JSON Schema Draft-07, define type 'object', "
            "allow additional properties, and incorporate the structure, types, constraints, and required fields as described."
        ),
        "parameters": {
            "type": "object",
            "properties": {
                "user_description": {
                    "type": "string",
                    "description": (
                        "A detailed description of the desired JSON schema. "
                        "Include information about the structure, types, constraints, and required fields. "
                        "For example: 'I need the JSON schema to have exactly 4 fields: weather, temperature in C, temperature in F, and location.'"
                    )
                }
            },
            "required": ["user_description"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

## Chain of Thoughts

In [ ]:
pf_task_chain_planner = PromptFunction(prompt = '''
You are given the following goal statement:
<<<task>>>

### Task:  
1. Decompose the goal statement into a list of main actionable tasks.  
2. Ensure each task is clearly defined, specifying its function and prerequisites.  
3. Provide a numbered list of tasks optimized for machine parsing, in plain text format, no markdown.  
4. Exclude any subtasks; focus solely on main tasks.  
5. If a task can be completed in a single action, return `<None>`.
                                       ''')

Placeholders: task


In [ ]:
plan = pf_task_chain_planner.execute(task="I have a txt file of finely curated stories text, but they are put together without index or separator. I want to let AI help split the txt file into each story, but the txt file is clearly over the max token limit. Can you suggest some ideas?",
                                     model='gpt-4o',
                                     temp = 1)
print(plan)

1. Analyze the txt file format to understand patterns that may indicate story boundaries, such as repetitive text, certain phrases, or length discrepancies.
2. Preprocess the txt file by breaking it into smaller, manageable chunks that fit within AI model token limits while maintaining potential story boundary indicators.
3. Utilize a text processing model to predict possible story boundaries within each chunk of text.
4. Compile the output predictions from the model to suggest potential splits for the entire text file.
5. Review and confirm the accuracy of suggested boundaries for final separation of stories, based on human verification or additional automated checks.


## GCP Vertex AI (Gemini)

In [ ]:
# Replace the `GOOGLE_CLOUD_PROJECT` and `GOOGLE_CLOUD_LOCATION` values
# with appropriate values for your project.
!GOOGLE_CLOUD_PROJECT='sonic-arcadia-373403'
!GOOGLE_CLOUD_LOCATION=us-central1
!GOOGLE_GENAI_USE_VERTEXAI=True

'GOOGLE_CLOUD_PROJECT' is not recognized as an internal or external command,
operable program or batch file.
'GOOGLE_CLOUD_LOCATION' is not recognized as an internal or external command,
operable program or batch file.
'GOOGLE_GENAI_USE_VERTEXAI' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
%pip install --upgrade --quiet google-genai pandas

In [16]:
from google import genai
from google.genai.types import Content, HttpOptions, Part
from dotenv import load_dotenv
from os import getenv

load_dotenv()
client = genai.Client(http_options=HttpOptions(api_version="v1"),
                      api_key=getenv("GCP_API_KEY"),)
chat = client.chats.create(
    model="gemini-2.0-flash-001",
    history=[
        Content(parts=[Part(text="Hello")], role="user"),
        Content(
            parts=[Part(text="Great to meet you. What would you like to know?")],
            role="model",
        ),
    ],
)
response = chat.send_message("write an haiku about the moon")
print(response.text)
# Example response:
# Okay, here's a story for you:
# ...

Silver orb ascends,
Guiding light in velvet skies,
Dreams in shadows dance.

